### 链接在数据库

In [7]:
import sys
import json
# sys.path.append('../')
from utils_ import *
# read_con = pymysql.connect(host="192.168.126.143",port = 3337,user='root', password='pjzhang', database='ccks_2019',charset='utf8')
# cur = read_con.cursor()

In [8]:
import re
def normalize(sentence):
    sentence = sentence.replace('•','·')
    # 年月日
    items = re.search(r"(\d{4}年\d{2}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-').replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
            
    items = re.search(r"(\d{4}年\d{1}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-0').replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{4}年\d{2}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-').replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{4}年\d{1}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-0').replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
            
    # 月日
    items = re.search(r"(\d{2}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{1}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = '0' + item.replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{1}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = '0' + item.replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{2}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
    # 年月
    items = re.search(r"(\d{4}年\d{2}月)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-').replace('月','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{4}年\d{1}月)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-0').replace('月','')
            sentence = sentence.replace(item,item_)
    return sentence

sents = ['歌手赵雷在2011年8月7日发行了哪首歌？','哪些人出生在9月19日？','村上春树所写的哪本小说是在2009年5月出版的？']
for sent in sents:
    print(normalize(sent))
    

歌手赵雷在2011-08-07发行了哪首歌？
哪些人出生在09-19？
村上春树所写的哪本小说是在2009-05出版的？


In [9]:
# 原始文件
def read_raw(fn):
    # 读取训练文件，返回问句，sql语句，答案的list
    question_list = []
    qindex_list = []
    sql_list = []
    answer_list = []
    with open(fn,'r',encoding='utf-8') as f:
        context=f.read()
    one_data=context.split('\n\n')
    for one in one_data:
        one=one.strip()
        if not one:
            continue
        question,sql,answer=one.split('\n')
        q_index = question[1:question.index(':')]
        question=question[question.index(':')+1:]
        question = normalize(question)
        sql=sql
        answer=[i.strip() for i in answer.strip().split('\t') if i.strip() ]
        question_list.append(question)
        qindex_list.append(q_index)
        sql_list.append(sql)
        answer_list.append(answer)
    return qindex_list,question_list,sql_list,answer_list

qindex_list,question_list,sql_list,answer_list = read_raw("data/test.txt")
question_list,sql_list,answer_list

(['"成败一知己，生死两妇人"所说的人物有什么重大成就？',
  '葬于茂陵的皇帝在位于哪段时间？',
  '"光武中兴"说的是哪位皇帝？',
  '李清照丈夫的代表作品是什么？',
  '"兰陵王"是对谁的尊称？',
  '《西厢记》又名什么？',
  '白炽灯是哪国人发明的？',
  '薛宝钗的哥哥外号叫什么？',
  '历史上的秦桧是哪里人？',
  '清华大学的校训是什么？',
  '严嵩的儿子人称？',
  '《西游记》中孙悟空的兵器叫什么？',
  '《红楼梦》中贾兰母亲的住所是哪里？',
  '唐太宗李世民的年号是什么？',
  '东坡肉的发明人死后谥号为？',
  '《红楼梦》中贾琏妻子的丫鬟是谁？',
  '乐山大佛始建于哪一年？',
  '木牛流马的发明者有哪些文学代表作？',
  '谁创立了关宁铁骑？',
  '明成祖朱棣的年号是什么？',
  '武则天的陵寝是？',
  '《红楼梦》中贾环的父亲的人物原型是谁？',
  '"负荆请罪"说的是谁的故事？',
  '战国四大名将之首的外号是？',
  '《六韬》的作者有什么重大成就？',
  '莫妮卡·贝鲁奇的代表作？',
  '《湖上草》是谁的诗？',
  '龙卷风的英文名是什么？',
  '新加坡的水域率是多少？',
  '商朝在哪场战役中走向覆灭？',
  '叔本华信仰什么宗教？',
  '大兴安岭的终点是哪里？',
  '演员梅艳芳有多高？',
  '被誉为万岛之国的是哪个国家？',
  '北京奥运会的口号是什么？',
  '新加坡的政治元老是谁？',
  '叶文洁毕业于哪个大学？',
  '"两弹一星"指的是什么？',
  '"超级碗"是什么体育项目联赛的年度冠军赛？',
  '清明节起源于哪里？',
  '诗经的内容分为哪三个部分？',
  '悉尼有哪些大学？',
  '凯文·杜兰特得过哪些奖？',
  '大连理工大学校歌是？',
  '清朝词人纳兰性德是哪个民族的人？',
  '戊戌变法中创建了什么报纸？',
  '东南大学的现任校长是谁？',
  '华山派的开山鼻祖是谁？',
  '庄周出生于哪里？',
  '三国女诗人蔡琰的主要作品有哪些？',
  '三国吕布，人称？',
  '《马可·波罗游记》的作者是哪国人？',
  '成吉思汗是对谁的尊称？',
  '我们尊称孙文先生为？

### 读取文件及处理函数

In [12]:
import re
def sql2ent(sql):
    #patterns=[re.compile(r'{\s?<(.*?)>'),re.compile(r' <(.*?)>\.'),re.compile(r'\.\s?<(.*?)>'),re.compile(r'"(.*)"')]
    entities=[]
    tmp=sql.split('where')
    if len(tmp)==2:
        sql=tmp[1]
    elif len(tmp)==1:
        sql=sql.split('WHERE')[1]
        
    #sql_part=sql.split('.')
    sql_part=re.split(r'(>|[a-z]|")\s?\.',sql)
    #print(sql_part)
    #patterns=[re.compile(r'<.*?>\s<(.*?)>\s?\.'),re.compile(r'\?\w\s<.*?>\s<(.*?)>'),re.compile(r'"(.*?)"')]
    #patterns=[re.compile(r'(<.*?>)\s<.*?>\s\?\s?'),re.compile(r'\?\w\s<.*?>\s(.*?)\s?\.'),re.compile(r"'(.*?)'"),re.compile(r'“(.*?)”')]
    #patterns=[re.compile(r'(<.*?>)\s<.*?>\s\?\s?'),re.compile(r'\?\w\s<.*?>\s(.*?)\s?\.')]
    p1 = re.compile(r'(<.*?>)\s<.*?>\s\?')  # <Y> <R> ?x
    p2 = re.compile(r'\?\w\s<.*?>\s(<.*)') # ?x <R> <Y>.
    p3 = re.compile(r'\?\w\s<.*?>\s(".*)')   # ?x <R> "" 
    p4 = re.compile(r'\?\w\s\?\w\s(<.*)')   # ?x ?x <A>
    p5 = re.compile(r'(<.*?>)\s\?\w\s(<.*)')   # <A> ?x <B>
    p6 = re.compile(r'(<.*?>)\s\?\w\s\?')   # <A> ?x ?x
    p7 = re.compile(r'filter regex\(\?\w, (".*?")\)')
    p8 = re.compile(r"filter regex\(\?\w, ('.*?')\)")
    patterns=[p1, p2, p3, p4, p5, p6, p7, p8]
    #print(sql_part)
    for sql_item in sql_part:
        for pattern in patterns:
            #tuples=pattern.findall(sql_item+'.')
            tuples=pattern.findall(sql_item)
            #print(sql_item+'.')
            #print('tuples',tuples)
            for es in tuples:
                if not isinstance(es,tuple):
                    entities.append(es)
                    #entities.append(es.split(' ')[-1])
                    continue
                for e in es:
                    entities.append(e)
    entities=list(set(entities))
    out=[]
    for e in entities:
        if not e:
            continue
        elif e[0]=='<' :
            if e[-1]!='>':
                out.append(e+'>')
            else:
                out.append(e)
        elif e[0]=='"' :
            if e[-1]!='"':
                out.append(e+'"')
            else:
                out.append(e)
        elif e[0]=="'":
            if e[-1]!="'":
                out.append(e+"'")
            else:
                out.append(e)
    return out

# def seq2ent2(sql):
#     entities=[]
#     sql=sql.split('where')[1]
#     p1 = re.compile(r'(<.*?>)\s<.*?>\s\?\w')  # <Y> <R> ?x
#     p2 = re.compile(r'\?\w\s<.*?>\s(<.*?>)\.') # ?x <R> <Y>.
#     p3 = re.compile(r'\?\w\s<.*?>\s"(.*?)"\.')   # ?x <R> "" 
#     p4 = re.compile(r'\?\w\s\?\w\s(<.*?>)\.')   # ?x ?x <A>
#     p5 = re.compile(r'(<.*?>)\s\?\w\s(<.*?>)\.')   # <A> ?x <B>
#     p6 = re.compile(r'(<.*?>)\s\?\w\s\?\w\.')   # <A> ?x ?x
#     p7 = re.compile(r'filter regex\(\?\w, "(.*?)"\)')
#     p8 = re.compile(r"filter regex\(\?\w, '(.*?)'\)")
#     patterns=[p1, p2, p3, p4, p5, p6, p7, p8]
#     for pattern in patterns:
#         tuples=pattern.findall(sql)
#         for es in tuples:
#             if not isinstance(es,tuple):
#                 entities.append(es)
#                 continue
#             for e in es:
#                 entities.append(e)
#     entities=list(set(entities))
#     #print(entities)
#     #entities=[e for e in entities if e and (e[0]=='<' or e[0]=='"') ]
#     out=[]
#     for e in entities:
#         if not e:
#             continue
#         elif e[0]=='<' :
#             if e[-1]!='>':
#                 out.append(e+'>')
#             else:
#                 out.append(e)
#         elif e[0]=='"' :
#             if e[-1]!='"':
#                 out.append(e+'"')
#             else:
#                 out.append(e)
#         elif e[0]=="'":
#             if e[-1]!="'":
#                 out.append(e+"'")
#             else:
#                 out.append(e)
#     return out

# sql里实体位置的内容
def sql2ent2(sql):
    entities=[]
    if 'where' in sql:
        sql=sql.split('where')[1]
    else:
        sql=sql.split('WHERE')[1]
    p1 = re.compile(r'{\s{0,2}(<.*?>)\s<.*?>\s\?\w\s?\.')  # <Y> <R> ?x
    p1_ = re.compile(r'\.\s{0,2}(<.*?>)\s<.*?>\s\?\w\s?\.')  # <Y> <R> ?x
    p2 = re.compile(r'\?\w\s<.*?>\s(<.*?>)\s?\.') # ?x <R> <Y>
    p3 = re.compile(r'\?\w\s<.*?>\s(".*?")\s?\.')   # ?x <R> "" 
    p4 = re.compile(r'\?\w\s\?\w\s(<.*?>)\s?\.')   # ?x ?x <A>
    p5 = re.compile(r'(<.*?>)\s\?\w\s(<.*?>)\s?\.')   # <A> ?x <B>
    p6 = re.compile(r'(<.*?>)\s\?\w\s\?\w\s?\.')   # <A> ?x ?x
    p7 = re.compile(r'filter regex\(\?\w, (".*?")\)')
    p8 = re.compile(r"filter regex\(\?\w, ('.*?')\)")
    p9 = re.compile(r"('.*?')")
    p10 = re.compile(r'(".*?")')
    patterns=[p1, p1_, p2, p3, p4, p5, p6, p7, p8, p9, p10]
    for pattern in patterns:
        tuples=pattern.findall(sql)
        #print(tuples)
        for es in tuples:
            if not isinstance(es,tuple):
                entities.append(es)
                continue
            for e in es:
                entities.append(e)
    entities=list(set(entities))
    #print(entities)
    #entities=[e for e in entities if e and (e[0]=='<' or e[0]=='"') ]
    out=[]
    for e in entities:
        if not e:
            continue
        elif e[0]=='<' :
            if e[-1]!='>':
                out.append(e+'>')
            else:
                out.append(e)
        elif e[0]=='"' :
            if e[-1]!='"':
                out.append(e+'"')
            else:
                out.append(e)
        elif e[0]=="'":
            if e[-1]!="'":
                out.append(e+"'")
            else:
                out.append(e)
        else:
            out.append(e)
    return out

# sql 里的有意义字符
def sql2seq(sql):
    tmp=sql.split('where')
    if len(tmp)==2:
        sql=tmp[1]
    elif len(tmp)==1:
        sql=sql.split('WHERE')[1]
    
    pattern=re.compile(r'(<.*?>|".*?"|\'.*?\')')
    items=pattern.findall(sql)
    return items
    
def entity_data(sql_list):
    entities_list=[]
    for sql in sql_list:
        entities=sql2ent2(sql)
        entities_list.append(entities)
    return entities_list

def rel_data(sql_list):
    tri_list=[]
    for sql in sql_list:
        tri=sql2seq(sql)
        tri_list.append(tri)
    return tri_list

In [11]:
sql2ent2('select ?x where { ?x <导演> <佩妮莱·费舍尔·克里斯滕森> . ?x <制片地区> <丹麦> . ?x <类型> <喜剧_（艺术体裁）> . }')

['<佩妮莱·费舍尔·克里斯滕森>', '<喜剧_（艺术体裁）>', '<丹麦>']

### 处理训练集

### 提取sql语句中有意义的字符串，包括头尾实体及关系，按sql语句中出现的顺序排列

In [9]:
mode = "test"# 'train','dev','test'
fn="data/%s.txt"%mode
qindex_list,q_list,s_list,a_list=read_raw(fn)

In [10]:
# 提取实体
e_list=entity_data(s_list)
e_list
not_found=0
not_found_index=[]
for index,i in enumerate(e_list):
    if not i:
        not_found+=1
        not_found_index.append(index)
    print(index,i)
print("not found:",not_found)
print("error question index:")
print(not_found_index)

0 ['<成败一知己，生死两妇人>']
1 ['<茂陵_（汉武帝陵寝）>']
2 ['<光武中兴>']
3 ['<李清照_（宋代女词人）>']
4 ['<兰陵王_（北齐徐州兰陵郡王）>']
5 ['<西厢记_（元代王实甫著古典戏剧）>']
6 ['<白炽灯>']
7 ['<薛宝钗>']
8 ['<秦桧_（南宋著名奸臣）>']
9 ['<清华大学>']
10 ['<严嵩>']
11 ['<孙悟空_（古典名著《西游记》中的主角）>']
12 ['<贾兰_（小说《红楼梦》人物）>']
13 ['<李世民_（唐太宗）>']
14 ['<东坡肉>']
15 ['<贾琏_（小说《红楼梦》人物）>']
16 ['<乐山大佛>']
17 ['<木牛流马_（诸葛亮发明的运粮工具）>']
18 ['<关宁铁骑>']
19 ['<朱棣_（明成祖）>']
20 ['<武则天_（中国武周时期女皇帝）>']
21 ['<贾环>']
22 ['<负荆请罪>']
23 ['"战国四大名将之首"']
24 ['<六韬>']
25 ['<莫妮卡·贝鲁奇>']
26 ['<湖上草>']
27 ['<龙卷风_（一种自然天气现象）>']
28 ['<新加坡>']
29 ['<商朝>']
30 ['<亚瑟·叔本华>']
31 ['<大兴安岭>']
32 ['<梅艳芳>']
33 ['"万岛之国"']
34 ['<2008年北京奥运会>']
35 ['<新加坡>']
36 ['<叶文洁>']
37 ['<两弹一星>']
38 ['<超级碗>']
39 ['<清明_（二十四节气之一）>']
40 ['<诗经>']
41 ['<悉尼_（澳大利亚新南威尔士州首府）>']
42 ['<凯文·杜兰特>']
43 ['<大连理工大学>']
44 ['<纳兰性德>']
45 ['<戊戌变法>']
46 ['<东南大学>']
47 ['<华山派_（道教支派）>']
48 ['"庄周"']
49 ['"蔡琰"']
50 ['<吕布_（东汉末年名将、割据诸侯）>']
51 ['<马可·波罗游记_（马可·波罗口述、鲁思梯谦记录游记）>']
52 ['<成吉思汗_（大蒙古国建立者）>']
53 ['"孙文"']
54 ['<雍正_（清世宗年号）>']
55 ['<玫瑰_（蔷薇科蔷薇属植物）>']
56 ['<富士山>']
57 ['<

In [11]:
# 提取关系
rel_list=rel_data(s_list)
rel_list
not_found=0
not_found_index=[]
for index,i in enumerate(rel_list):
    if not i:
        not_found+=1
        not_found_index.append(index)
    print(index,i)
print("not found:",not_found)

0 ['<成败一知己，生死两妇人>', '<主要人物>', '<主要成就>']
1 ['<陵墓>', '<茂陵_（汉武帝陵寝）>', '<在位时间>']
2 ['<光武中兴>', '<相关人物>']
3 ['<李清照_（宋代女词人）>', '<配偶>', '<代表作品>']
4 ['<兰陵王_（北齐徐州兰陵郡王）>', '<本名>']
5 ['<西厢记_（元代王实甫著古典戏剧）>', '<又名>']
6 ['<白炽灯>', '<发明者>', '<国籍>']
7 ['<妹妹>', '<薛宝钗>', '<别名>']
8 ['<秦桧_（南宋著名奸臣）>', '<出生地>']
9 ['<清华大学>', '<校训>']
10 ['<严嵩>', '<儿子>', '<别称>']
11 ['<孙悟空_（古典名著《西游记》中的主角）>', '<兵器>']
12 ['<贾兰_（小说《红楼梦》人物）>', '<母亲>', '<居所>']
13 ['<李世民_（唐太宗）>', '<年号>']
14 ['<东坡肉>', '<创制人>', '<谥号>']
15 ['<丈夫>', '<贾琏_（小说《红楼梦》人物）>', '<丫鬟>']
16 ['<乐山大佛>', '<开凿时间>']
17 ['<木牛流马_（诸葛亮发明的运粮工具）>', '<发明者>', '<代表作品>']
18 ['<关宁铁骑>', '<创始人>']
19 ['<朱棣_（明成祖）>', '<年号>']
20 ['<武则天_（中国武周时期女皇帝）>', '<陵寝>']
21 ['<贾环>', '<父亲>', '<人物原型>']
22 ['<典故>', '<负荆请罪>']
23 ['<地位>', '"战国四大名将之首"', '<别称>']
24 ['<六韬>', '<作者>', '<主要成就>']
25 ['<莫妮卡·贝鲁奇>', '<代表作品>']
26 ['<主要作品>', '<湖上草>']
27 ['<龙卷风_（一种自然天气现象）>', '<外文名>']
28 ['<新加坡>', '<水域率>']
29 ['<商朝>', '<灭亡>']
30 ['<亚瑟·叔本华>', '<信仰>']
31 ['<大兴安岭>', '<终点>']
32 ['<梅艳芳>', '<身高>']
33 ['<誉称>', '"万岛之国"']
34 ['<2

In [ ]:
# 提取mention
def load_mention(cur,q_list,e_list):
    mention_list=[]
    for index,es in enumerate(e_list):
        mention=[]
        if not es:
            mention_list.append([])
            continue
        for item in es:
            if item[0]=='"' or item[0]=="'":
                if item[1:-1] in q_list[index]:
                    mention.append(item[1:-1])
            else:
                cand=search_alias(item[1:-1],cur)
                cand.append(del_des(item)[1:-1])
                cand.append(item[1:-1])
                tmp=[c for c in cand if c in q_list[index]]
                tmp=sorted(tmp,key=lambda x:len(x),reverse=True)
#             from ipdb import set_trace
#             set_trace()
                if not tmp:
                    mention.append([])
                else:
                    mention.append(tmp[0])
        mention_list.append(mention)
    return mention_list

mention_list=load_mention(cur,q_list,e_list)

In [ ]:
num_unuse=0
for i, items in enumerate(mention_list):
    if not items or [] in items:
        num_unuse+=1
        print(q_list[i],s_list[i],e_list[i])
        print('-'*30)
num_unuse

In [ ]:
# patternHead=re.compile(r'(\?[a-z]) <.*?> "|<')
# patternHead=re.compile(r'\?[a-z] <')
# patternHead.findall('{ <青城派_（金庸小说中的门派）> ?x <掌门人>  . ?x <号称> "三峡以西剑法第一" . }')
del_des('<舒杰_(维力医疗高管)>')

### 问句根据sparql进行分类

In [ ]:
import re
fn_out="data/classes_questions/%s"%mode+"_问句类型%s.txt"
patternVar=re.compile(r'(\?[a-z])')
patternHead=re.compile(r'(\?[a-z]) <')
patternRel=re.compile(r'> (\?[a-z]) <')
patternTail=re.compile(r'(\?[a-z])(?:\.| \.)')

p7 = re.compile(r'filter regex\(\?\w, (".*?")\)')
p8 = re.compile(r"filter regex\(\?\w, ('.*?')\)")

num_q=0
num_one=0
num_one_head=0 # 尾->头
num_one_rel=0 # 找关系
num_one_tail=0 # 头->尾
num_two=0
num_three=0
num_multi=0
num_two_tail0=0 # 样式一（双尾找一头）
num_two_tail1=0 # 样式二（找中间）
num_two_tail2=0 # 样式三（双头找一尾）
pattern1=0 # 样式四（链式找尾实体）
pattern2=0 # 样式六（三尾找一头）
pattern3=0 # 样式七（三头找一尾）
pattern4=0 # 样式五（尾-头-尾)
pattern_new=0 # 新样式（头尾头）
f_type=''
data=''
with open(fn,'r',encoding='utf-8') as fp:
    for line in fp:
        
        if line[0].lower()=='s':
            line=line.strip()
            if 'where' in line:
                line=line.split('where')
                prefix = line[0]
                line=line[1]
            elif 'WHERE' in line:
                line=line.split('WHERE')
                prefix = line[0]
                line=line[1]
            else:
                print('error')
#             data+=line.strip()
#             data+='\n'
            num_q+=1
            
#             if num_q ==2297:
#                 from pdb import set_trace
#                 set_trace()
            
            l=patternVar.findall(line)
            count=len(l)
            
            if len(patternVar.findall(prefix))>1:
                f_type="0"
            
            elif p7.findall(line) or p8.findall(line) or 'UNION' in line or 'union' in line or 'filter' in line or 'regex' in  line:
                num_multi+=1
                f_type="0"
                
            elif count==1:
                num_one+=1
                num_e = len(patternRel.findall(line))
                if num_e == 1:
                    num_one_rel += 1
                    f_type='3' #"两头找关系"
                    continue
                num_e=len(patternHead.findall(line))
                if num_e==1:
                    num_one_head+=1
                    f_type='2' #"尾找头"
                    continue
                num_e=len(patternTail.findall(line))
                if num_e==1:
                    num_one_tail+=1
                    f_type='1' #"头找尾"
                    continue
            elif count==2:
                num_tail=len(patternTail.findall(line))
                num_two+=1
                if num_tail==0:
                    num_two_tail0+=1
                    f_type='6' #"双尾找一头"
                elif num_tail==1:
                    num_two_tail1+=1
                    f_type='8' #"样式二（找中间）"
                elif num_tail==2:
                    num_two_tail2+=1
                    f_type='9' #"样式三（双头找一尾）"
            elif count==3:
                num_three+=1
                num_tail = len(patternTail.findall(line))
                num_head = len(patternHead.findall(line))
                if len(set(l)) == 1:
                      f_type='10'
                elif num_head==1 and num_tail==2:
                    pt=patternTail.findall(line)
                    ph=patternHead.findall(line)
                    if pt[0]==pt[1]:
                        pattern_new+=1
                        f_type='5' #"新样式（头尾头）"
                    else:
                        pattern1+=1
                        f_type='4' #"样式四（链式找尾实体）"
                elif num_head==3:
                    pattern2+=1
                    f_type='10' #"样式六（三尾找一头）"
                elif num_tail==3:
                    pattern3+=1
                    f_type='10' #"样式七（三头找一尾）"
                if num_head==2 and num_tail==1:
                    pattern4+=1
                    f_type='7' #"样式五（尾-头-尾）"
            else:
                num_multi+=1
                f_type="0"
            
        elif line[0].lower()=="q":
            data+=line.strip()
            pass
        elif not line.strip():
            data+="\t%s\n"%f_type
            with open(fn_out%f_type,'a',encoding='utf-8') as f_out:
                f_out.write(data)
                data=''

### 提取句子类型数据

In [ ]:
# 提取句子类型数据
def load_type(num_case,num_type):
    type_list=[0]*num_case
    for i in range(1,num_type):
        cur_fn = fn_out%i
        if not os.path.exists(cur_fn):
            continue
        with open(cur_fn,'r') as f:
            for line in f:
                line=line.strip()
                if line:
                    index=int(line[1:line.index(":")])-1
                    type_list[index]=i
    return type_list
type_list=load_type(len(mention_list),11)
assert len(q_list)==len(type_list)
type_list

### 保存数据

In [ ]:
# def get_dict(qindex=None,q=None,sql=None,ans=None,ent=None,rel=None,mention=None,qtype=None):
    

In [ ]:
# 保存所有问句集中提取的信息
import json
assert(len(qindex_list)==len(e_list))
assert(len(q_list)==len(e_list))
assert(len(q_list)==len(rel_list))
assert(len(q_list)==len(mention_list))
all_data=(qindex_list,q_list,s_list,a_list,e_list,rel_list,mention_list,type_list)
with open("data/%s.json"%mode,'w')as f:
    json.dump(all_data,f,ensure_ascii=False)
print("output %s successful!" % mode)

### 生成NER训练文件

In [9]:
import numpy as np
import random
import json
#path = 
modes=["train","valid",'test']
fn_out="data/%s_bert_ner_input.txt"

for mode in modes:
    fn = "data/%s.json"%mode
    with open(fn,'r') as f:
        cur_data = json.load(f)
        question_label=[]
        for q,ms in zip(cur_data[1],cur_data[6]):
            if not ms or [] in ms:
                continue
            flag=False
            for i in ms:
                if isinstance(i,list):
                    flag=True
            if flag:
                continue

            label=['O']*len(q)
            for m in ms:
                start=q.index(m)
                label[start]='B-LOC'
                if len(m)>1:
                    for it in range(start+1,start+len(m)):
                        label[it]='I-LOC'
            question_label.append((q,label))

        with open(fn_out%mode,'w',encoding='utf-8') as f:
            for q,l in question_label:
                for a,b in zip(q,l):
                    f.write("%s %s\n"%(a,b))
                f.write('   \n')

### 生成test文件

In [ ]:
mode = 'test' 
fn = "%s.json"%mode
with open(fn,'r') as f:
    cur_data = json.load(f)
    questions_test = cur_data[1]

In [ ]:
fn_out_test = fn_out%mode
with open(fn_out_test,'w',encoding='utf-8') as fp:
    for line in questions_test:
        for word in line:
            fp.write("%s\tO\n"%(word))
        fp.write("\n")

## 测试cell

In [7]:
import json
mode = 'train' 
fn = "data/%s.json"%mode
with open(fn,'r') as f:
    cur_data = json.load(f)
    questions_test = cur_data[1]
    mentions_test = cur_data[-2]
    sql = cur_data[2]
for q,m,s in zip(questions_test,mentions_test,sql):
    if not m or [] in m:
        #print(q,'\t',m,'\t',s)
        pass
    else:
        print(q,'\t',m,'\t',s)

莫妮卡·贝鲁奇的代表作？ 	 ['莫妮卡·贝鲁奇'] 	 select ?x where { <莫妮卡·贝鲁奇> <代表作品> ?x. }
《湖上草》是谁的诗？ 	 ['湖上草'] 	 select ?x where { ?x <主要作品> <湖上草>. }
龙卷风的英文名是什么？ 	 ['龙卷风'] 	 select ?x where { <龙卷风_（一种自然天气现象）> <外文名> ?x. }
新加坡的水域率是多少？ 	 ['新加坡'] 	 select ?x where { <新加坡> <水域率> ?x. }
商朝在哪场战役中走向覆灭？ 	 ['商朝'] 	 select ?x where { <商朝> <灭亡> ?x. }
叔本华信仰什么宗教？ 	 ['叔本华'] 	 select ?y where { <亚瑟·叔本华> <信仰> ?y. }
大兴安岭的终点是哪里？ 	 ['大兴安岭'] 	 select ?x where { <大兴安岭> <终点> ?x. }
演员梅艳芳有多高？ 	 ['梅艳芳'] 	 select ?y where { <梅艳芳> <身高> ?y. }
被誉为万岛之国的是哪个国家？ 	 ['万岛之国'] 	 select ?x where { ?x <誉称> "万岛之国". }
北京奥运会的口号是什么？ 	 ['北京奥运会'] 	 select ?x where { <2008年北京奥运会> <口号> ?x. }
新加坡的政治元老是谁？ 	 ['新加坡'] 	 select ?x where { <新加坡> <政治元老> ?x. }
叶文洁毕业于哪个大学？ 	 ['叶文洁'] 	 select ?x where { <叶文洁> <毕业院校> ?x. }
"两弹一星"指的是什么？ 	 ['两弹一星'] 	 select ?x where { <两弹一星> <原指> ?x. }
"超级碗"是什么体育项目联赛的年度冠军赛？ 	 ['超级碗'] 	 select ?x where { <超级碗> <介绍> ?x. }
清明节起源于哪里？ 	 ['清明节'] 	 select ?x where { <清明_（二十四节气之一）> <起源> ?x. }
诗经的内容分为哪三个部分？ 	 ['诗经'] 	 select ?x where { <诗经> <

In [11]:
import re
import json
with open('/home/ubuntu/Desktop/TransferNet-master/NLPCC-KB/data/nlpcc2016-2018.kbqa.train', 'r', encoding='utf-8') as file:
    lines = file.readlines()
data = {}
answer = {}
# 遍历每四行文本，提取信息并填充到JSON对象中
for i in range(0, len(lines), 3):
    te = lines[i].strip('\n').split('>')[1].strip('\t').split(' ||| ')
    if  te[2] == '':
        continue
    te = [i.strip().replace(u'\xa0', '') for i in te]
    ques = lines[i+1].strip('\n').split('>')[1].strip()
    data[ques.replace(' ','')] = te[0]
    answer[ques.replace(' ','')] = [te[2]]
newfile = open('/home/ubuntu/Desktop/TransferNet-master/NLPCC-KB/data/train2016-2018.json', 'w', encoding='utf-8')
with open('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/data/cner/raw_data/train2016.char.bmes', 'r', encoding='utf-8') as fp:
    lines = fp.readlines()
    texts = []
    entities = []
    words = []
    entity_tmp = []
    entities_tmp = []
    flag = False
    for i, line in enumerate(lines):
        line = line.strip().split(" ")
        if len(line) == 2:
            word = line[0]
            label = line[1]
            words.append(word)
            if "B-" in label:
                entity_tmp.append(word)
                flag = True
            elif "I-" in label:
                entity_tmp.append(word)
                if i < len(lines) - 1 and len(lines[i + 1].strip().split(" ")) != 2:
                    if ("".join(entity_tmp)) not in entities_tmp:
                        entities_tmp.append("".join(entity_tmp))
                    entity_tmp = []
            elif "O" == label and flag:
                flag = False
                if ("".join(entity_tmp)) not in entities_tmp:
                    entities_tmp.append("".join(entity_tmp))
                entity_tmp = []
        else:
            texts.append("".join(words))
            entities.append(entities_tmp)
            words = []
            entities_tmp = []
# {"text_id": "1", "text": "草本植物的含水量为70%～85%，木本植物的含水量稍低于草本植物", "mention_data": [{"kb_id": "383066", "mention": "草本植物", "offset": 0}, {"kb_id": "411648", "mention": "含水量", "offset": 5}, {"kb_id": "421434", "mention": "木本植物", "offset": 17}]}
for i, text in enumerate(texts):
    js = {'text_id':i + 1, 'text': text, 'mention_data':[],'answer':[]}
    for ent in entities[i]:
        ent = re.escape(ent)
        for span in re.finditer(ent, text):
            start = span.start()
            if text in data.keys():
                temp = {'kb_id':data[text],'mention':ent.replace('\\', ''),'offset':start}
                js['mention_data'].append(temp)
            # else:
            #     print(text)
    if len(js['mention_data']) != 0:
        js['answer'] = answer[text]
        json.dump(js,newfile,ensure_ascii=False)
        newfile.write('\n')


In [28]:
import json
mode = 'test' 
fn = "data/%s.json"%mode
newfile = open('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/data/cner/raw_data/test2019.json', 'w', encoding='utf-8')
with open(fn,'r') as f:
    cur_data = json.load(f)
    questions_test = cur_data[1]
    ans_test = cur_data[3]
    mentions_test = cur_data[-2]
    sql = cur_data[2]
total = len(sql) - 1
cnt = 0
for q,m,s,a in zip(questions_test,mentions_test,sql,ans_test):
    print(cnt,total)
    if not m or [] in m:
        #print(q,'\t',m,'\t',s)
        pass
    else:
        # print(q,'\t',m,'\t',s)
        js = {'text_id':cnt + 1, 'text': q, 'mention_data':[], 'answer':[]}
        sqs = sql2ent2(s)
        for ent in m:
            ent = re.escape(ent)
            for span in re.finditer(ent, q):
                start = span.start()
                for sq in sqs:
                    if ent in sq:
                        temp = {'kb_id':sq.replace('<', '').replace('>','').replace('"','').strip(),'mention':ent.replace('\\', ''),'offset':start}
                        js['mention_data'].append(temp)
                        break
        if len(js['mention_data']) != 0:
            js['answer'] = [i.replace('<','').replace('>','').replace('"','').strip() for i in a]
            json.dump(js,newfile,ensure_ascii=False)
            newfile.write('\n')
            cnt +=1
        

0 765
1 765
2 765
3 765
4 765
5 765
5 765
6 765
7 765
8 765
9 765
10 765
11 765
12 765
13 765
14 765
15 765
16 765
17 765
18 765
19 765
20 765
21 765
22 765
23 765
23 765
24 765
25 765
26 765
27 765
28 765
29 765
30 765
31 765
32 765
33 765
34 765
35 765
36 765
37 765
37 765
38 765
39 765
40 765
41 765
42 765
43 765
44 765
45 765
46 765
47 765
48 765
48 765
49 765
50 765
51 765
52 765
53 765
54 765
55 765
56 765
57 765
58 765
59 765
60 765
61 765
62 765
63 765
64 765
65 765
66 765
67 765
68 765
69 765
70 765
71 765
72 765
73 765
74 765
75 765
76 765
77 765
77 765
78 765
79 765
80 765
81 765
82 765
83 765
84 765
85 765
86 765
87 765
88 765
89 765
90 765
91 765
92 765
93 765
94 765
95 765
96 765
96 765
97 765
98 765
99 765
100 765
101 765
102 765
103 765
104 765
105 765
106 765
107 765
108 765
109 765
110 765
111 765
112 765
113 765
114 765
115 765
115 765
115 765
116 765
117 765
118 765
118 765
118 765
119 765
120 765
121 765
122 765
122 765
123 765
123 765
124 765
125 765
126 765
127 7

In [28]:
with open('/home/ubuntu/Desktop/TransferNet-master/CCKS-KB/data/triple.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    total = len(lines) - 1
    for i, line in enumerate(lines):
        if i < 10:
            print(line)

<美国奥可斯（香港）国际控股集团>	<类型>	<文学作品> .

<美国奥可斯（香港）国际控股集团>	<类型>	<文化> .

<寻美中国>	<类型>	<品牌> .

<青春是我和你一杯酒的深>	<类型>	<文学作品> .

<青春是我和你一杯酒的深>	<类型>	<网络小说> .

<青春是我和你一杯酒的深>	<类型>	<小说作品> .

<青春是我和你一杯酒的深>	<类型>	<小说> .

<青春是我和你一杯酒的深>	<类型>	<娱乐作品> .

<青春是我和你一杯酒的深>	<类型>	<书籍> .

<青春是我和你一杯酒的深>	<类型>	<中国文学> .



In [30]:
line = '<寻美中国>	<类型>	<品牌> .'
line = line.split('.')[0].strip()
print(line.split('	'))

['<寻美中国>', '<类型>', '<品牌>']


In [21]:
import re
import json
from tqdm import tqdm
pattern = re.compile(r'[\t ]+')
count = 0
other = 0
f =  open('/home/ubuntu/Desktop/TransferNet-master/CCKS-KB/data/triple.txt','r', encoding = 'utf-8').readlines()
triples = {}
for line in tqdm(f):
    if line.strip()[-1] == ".": 
        temp = line.strip()[0:-1].strip()
    else:
        print("error!",line)
        continue
    if(temp.count('\t') != 2):
        temp = re.sub(pattern, '\t', temp)
    count += 1
    if(temp.count('\t') != 2):
        print(line, "因为格式不符被丢弃")
        other += 1
        continue
    s = temp.split('\t')[0].replace('<', '').replace('>','').replace('"','').strip()
    p = temp.split('\t')[1].replace('<', '').replace('>','').replace('"','').strip()
    o = temp.split('\t')[2].replace('<', '').replace('>','').replace('"','').strip()
    if s == '' or p == '' or o == '':
        continue
    if s in triples.keys():
        triples[s][p] = o
    else:
        triples[s] = {p:o}
print('共有数据：', count)
print('被丢弃数据：', other)
with open('/home/ubuntu/Desktop/TransferNet-master/CCKS-KB/data/knowledge.json', 'w', encoding='utf-8') as fi:
    json.dump(triples,fi,ensure_ascii=False)

 99%|█████████▉| 66269036/66630393 [01:38<00:00, 626195.22it/s]

<外汇风险><类型>	"交易风险" .
 因为格式不符被丢弃
<外汇风险><类型>	"评价风险" .
 因为格式不符被丢弃
<外汇风险><类型>	"经济风险" .
 因为格式不符被丢弃
<外汇风险><类型>	"储备风险" .
 因为格式不符被丢弃
<维力医疗>	<公司法人>  <向彬_ (维力医疗董事长)> .
 因为格式不符被丢弃


100%|██████████| 66630393/66630393 [01:39<00:00, 669803.77it/s]

<人为性皮炎>	<openkg_uri>	"http://www.openkg.cn/COVID-19/wiki/resource/R11270" .<布丁股份>	<董事长>	<朱晖>.
 因为格式不符被丢弃
共有数据： 66630393
被丢弃数据： 6


In [2]:
import jieba
word_list=list(jieba.cut("成人呼吸窘迫综合征。",cut_all=False))
print(word_list)
# import jieba
# import tools
# import os
# from transformers import BertTokenizer
# word2id, _ = tools.load_vocab()

# def cut_word(sentence,tokenizer):

#     word_list=list(jieba.cut(sentence,cut_all=True))
#     print(word_list)
#     token_list = ['[CLS]']+tokenizer.tokenize(sentence)+['[SEP]']
#     win=['。' for i in range(6)]
#     token_list = win + token_list + win

#     char_word=[]
#     for i,token in enumerate(token_list[6:-6]):
#         char_word_list =[]
#         sub_text=''.join(token_list[i:i+12])
#         for word in word_list:
#             if token in word and word in sub_text:
#                 char_word_list.append(word)
#         char_word_id=[0,0,0,0] # [B,M,E,S]
#         for word in char_word_list:
#             if token == word and word in word2id:
#                 char_word_id[3]=word2id[word]
#             else:
#                 index=word.index(token)
#                 if index==0 and word in word2id:
#                     char_word_id[0]=word2id[word]
#                 elif index==len(word)-1 and word in word2id:
#                     char_word_id[2]=word2id[word]
#                 elif word in word2id:
#                     char_word_id[1]=word2id[word]
#         char_word.append(char_word_id)

#     while len(char_word)<80:
#         char_word.append([0 for i in range(4)])
#     return char_word[:80]
# tokenizer = BertTokenizer(os.path.join('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/model_hub/chinese-bert-wwm-ext', 'vocab.txt'))
# print(cut_word('中国语言学', tokenizer))

# import pickle
# with open(os.path.join('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/', '{}.pkl'.format('data_params')), 'rb') as f:
#         data = pickle.load(f)
# print(data)

['成人', '呼吸', '窘迫', '综合征', '。']


In [11]:
# import json
# with open('/media/ubuntu/A4CAA455CAA42610/BLINK-main/blink/biencoder/data/kbqa/ccks2019/kb.json', 'r', encoding='utf-8') as f:
#     dic = json.load(f)

# cnt = 1
# tem = {}
# for k in dic.keys():
#     k_t = k.split('_')
#     if len(k_t) > 1 and len(k_t[1].split('（')) > 1:
#         k = k_t[0]
#     if k not in tem:
#         tem[k] = cnt
#         cnt += 1
# with open('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/data/vocab.json', 'w', encoding='utf-8') as f:
#     json.dump(tem, f, ensure_ascii=False)
import json
with open('/media/ubuntu/A4CAA455CAA42610/BLINK-main/blink/biencoder/data/kbqa/nlpcc2016/kb.json', 'r', encoding='utf-8') as f:
    dic = json.load(f)

cnt = 1
tem = {}
for k in dic.keys():
    if k[0] != '(':
        k_t = k.split('(')
        if len(k_t) > 1:
            k = k_t[0]
    if k not in tem:
        tem[k] = cnt
        cnt += 1
with open('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/data/vocab.json', 'w', encoding='utf-8') as f:
    json.dump(tem, f, ensure_ascii=False)

In [9]:
import json
with open('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/data/cner/final_data/ccks2019_lex/vocab_jieba.json', 'r', encoding='utf-8') as f:
    dic_jie = json.load(f)
with open('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/data/cner/final_data/ccks2019_lex/vocab_kb.json', 'r', encoding='utf-8') as f:
    dic_kb = json.load(f)

cnt = 1
tem = {}
for k in dic_jie.keys():
    if k not in tem:
        tem[k] = cnt
        cnt += 1
for k in dic_kb.keys():
    if k not in tem:
        tem[k] = cnt
        cnt += 1


with open('/home/ubuntu/Desktop/pytorch_bert_bilstm_crf_ner-main/data/vocab.json', 'w', encoding='utf-8') as f:
    json.dump(tem, f, ensure_ascii=False)

In [3]:
import torch
A = torch.Tensor([[1,2,3],[1,2,3],[4,5,6]])
B = torch.Tensor([[7,7,7],[8,8,8],[9,9,9]])
word_emb = torch.mul(A,B)
print(word_emb)

tensor([[ 7., 14., 21.],
        [ 8., 16., 24.],
        [36., 45., 54.]])
